In [2]:
import pandas as pd

Tu so zbrani vsi podatki.

In [5]:
podatki = pd.read_csv('zbrani_podatki.csv')
pd.options.display.max_rows = 20
podatki.tail(100)

,LETO,AVTOR,MESTO,DRŽAVA,TABLATURA,INSTRUMENT,ŠTEVILO (SKUPIN) STRUN,TREATISE
9304,NONE,NONE,Vienna,Austria,NONE,guitar,NONE,NO
9305,NONE,NONE,Vienna,Austria,NONE,guitar,NONE,NO
9306,NONE,NONE,NONE,NONE,NONE,guitar,NONE,NO
9307,NONE,NONE,Dessau,Germany,NONE,guitar,NONE,NO
9308,NONE,NONE,Vienna,Austria,NONE,guitar,NONE,NO
...,...,...,...,...,...,...,...,...
9399,NONE,NONE,Leipzig and Mainz,Germany,NONE,guitar,NONE,NO
9400,NONE,NONE,Hamburg and Altona,Germany,NONE,guitar,NONE,NO
9401,NONE,NONE,Vienna,Austria,NONE,guitar,NONE,NO
9402,NONE,NONE,Bonn,Germany,NONE,guitar,NONE,NO
